<a href="https://colab.research.google.com/github/ShubhamZoro/PDFQA/blob/main/PdfQueryLangchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PDF Query Using Langchain

In [ ]:
!pip install langchain
!pip install openai
!pip install pypdf
!pip install chromadb
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all th

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [ ]:
# provide the path of  pdf file/files.
# pdfreader = PdfReader('48lawsofpower.pdf')
loader = PyPDFLoader("48lawsofpower.pdf")
pages = loader.load()


In [ ]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(

    chunk_size = 800,
    chunk_overlap  = 200,

)
texts = text_splitter.split_documents(pages)

In [ ]:
len(texts)

17

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
document_search = Chroma.from_documents(texts, embeddings)

In [ ]:
document_search


In [ ]:
chain=RetrievalQA.from_chain_type(OpenAI(),retriever=document_search.as_retriever(),chain_type="stuff")
query="Can you give me an example from history where the enemy was crushed totally from the book"
chain.invoke(query)


{'query': 'Can you give me an example from history where the enemy was crushed totally from the book',
 'result': ' One example from the book is the story of Empress Wu of China who ruthlessly eliminated anyone who posed a threat to her reign, including her own sons. She was able to maintain her power for decades until she was eventually forced to abdicate.'}

In [ ]:
query="Can you give me an example from history where the enemy was crushed totally from the book"
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
document_search = Chroma.from_documents(texts, embeddings)
docs=document_search.similarity_search(query)
vectordb=Chroma.from_documents(documents=docs,embedding=embeddings)
chain = RetrievalQA.from_chain_type(OpenAI(), retriever=vectordb.as_retriever(),chain_type="stuff")
chain.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' One example from the book is the story of Emperor Augustus who used the "crushing" tactic against his enemy Mark Antony. Augustus pretended to offer peace to Antony, but then used the opportunity to attack and defeat him, ultimately leading to his downfall and death.'

In [ ]:
persist_dir="embedd"
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb= Chroma.from_documents(documents=docs, embedding=embeddings,persist_directory=persist_dir)
vectordb.persist()

In [ ]:
vectordb=None
persist_dir="embedd"
embedding=OpenAIEmbeddings(model="text-embedding-3-large")
document_search=Chroma(persist_directory=persist_dir,embedding_function=embedding)
chain = RetrievalQA.from_chain_type(OpenAI(), retriever=document_search.as_retriever(),chain_type="stuff")
query = "What's the point of making myself less accessible"
list1=chain.run(query)



In [ ]:
list1

' The point is to create an air of mystery and desirability around yourself. By making yourself less accessible, you become more valuable and sought after. This can also give you more control in relationships and negotiations.'